In [1]:
import os

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

In [2]:
directory_in = './file/3.part'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input = df_input[df_input['Vehicle Url'] != ''].reset_index(drop=True)

df_input['No'] = df_input['No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：1199



,No,Type,Part Number,Brand,Description,Application Summary,OEM,Picture,Url,Json_Src,Vehicle Url,Json_Specification
0,1,Brake Backing Plate,924-212,Dorman - OE Solutions,Brake Backing Plate - 1 Pair,Ford 2012-05,5C3Z-2B636-AA;5C3Z-2B636-BA;8C3Z-2209-A;8C3Z-2...,,https://www.dormanproducts.com/p-45360-924-212...,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Brake Type"": ""Disc""}, ""1"": {""Color"": ""..."
1,2,Brake Backing Plate,924-218,Dorman - OE Solutions,Brake Backing Plate - 1 Pair,"Chevrolet 1999-88, GMC 1999-88",18013180;18013181,,https://www.dormanproducts.com/p-49987-924-218...,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Brake Type"": ""Drum""}, ""1"": {""Color"": ""..."
2,3,Brake Backing Plate,924-219,Dorman - OE Solutions,Brake Backing Plate - 1 Pair,Jeep Grand Cherokee 2004-99,5011982AA,,https://www.dormanproducts.com/p-49997-924-219...,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Brake Type"": ""Disc""}, ""1"": {""Color"": ""..."
3,4,Brake Backing Plate,924-220,Dorman - OE Solutions,Brake Backing Plate - 1 Pair,"Buick 1987-78, Chevrolet 1992-78, GMC 1992-78,...",18013524;18013525,,https://www.dormanproducts.com/p-50277-924-220...,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Brake Type"": ""Drum""}, ""1"": {""Color"": ""..."
4,5,Brake Backing Plate,924-234,Dorman - OE Solutions,Brake Backing Plate - 1 Pair,"Chevrolet 2020-06, GMC 2020-06, Hummer 2009-06",15949892;85566024,,https://www.dormanproducts.com/p-137413-924-23...,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Brake Type"": ""Disc""}, ""1"": {""Circular""..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1194,1216,Wheel Lug Stud,610-746.1,Dorman - AutoGrade - Bagged,"M14-1.50 Serrated Wheel Stud - 0.62in Knurl, 5...","Ford GT 2021-17, Ford Mustang 2021-19, Lincoln...",DE9Z1107A,,https://www.dormanproducts.com/p-154062-610-74...,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Bolt Grade"": ""10.9""}, ""1"": {""Color"": ""..."
1195,1217,Wheel Lug Stud,610-785,Dorman - AutoGrade - Boxed,Heavy Duty Wheel Stud,"Ram 3500 2024-16, 2014-12, Ram 4000 2024-12, R...",6509853AA;6513762AA,,https://www.dormanproducts.com/p-139057-610-78...,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Bolt Grade"": ""10.9""}, ""1"": {""Color"": ""..."
1196,1218,Wheel Lug Stud,610-785.1,Dorman - AutoGrade - Bagged,"M14-1.50 Serrated Wheel Stud - 15.79mm Knurl, ...","Ram 3500 2024-16, 2014-12, Ram 4000 2024-12, R...",6509853AA;6513762AA,,https://www.dormanproducts.com/p-139056-610-78...,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Bolt Grade"": ""10.9""}, ""1"": {""Color"": ""..."
1197,1219,Wheel Lug Stud,610-789,Dorman - AutoGrade - Boxed,"M14-1.50 Serrated Wheel Stud - 064in Knurl, 46...","Infiniti 2024-11, Nissan 2024-12",432221LB0A,,https://www.dormanproducts.com/p-154073-610-78...,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Bolt Grade"": ""10.9""}, ""1"": {""Color"": ""..."


In [4]:
(crawler_index, crawler_series) = next(df_input.iterrows())

crawler_index, crawler_series

(0,
 No                                                                     1
 Type                                                 Brake Backing Plate
 Part Number                                                      924-212
 Brand                                              Dorman - OE Solutions
 Description                                 Brake Backing Plate - 1 Pair
 Application Summary                                         Ford 2012-05
 OEM                    5C3Z-2B636-AA;5C3Z-2B636-BA;8C3Z-2209-A;8C3Z-2...
 Picture                                                                 
 Url                    https://www.dormanproducts.com/p-45360-924-212...
 Json_Src               {"0": "https://static.dormanproducts.com/image...
 Vehicle Url            https://www.dormanproducts.com/itemdetailapp.a...
 Json_Specification     {"0": {"Brake Type": "Disc"}, "1": {"Color": "...
 Name: 0, dtype: object)

In [5]:
list_dict_correct_temp, page = [], 0
while True:

    count_retry = 0
    while True:
        count_retry += 1
        print(page + 1, count_retry)
        try:
            url_request = f'''{crawler_series['Vehicle Url']}&num=50&start={page * 50}'''

            resp = requests.get(url_request,
                                headers=get_header(),
                                proxies=get_proxy(),
                                timeout=(5, 15))

            if resp.status_code == 200:
                soup = BeautifulSoup(resp.text, 'lxml')
                html = etree.HTML(str(soup))

                if html.xpath('//div[@class="detail-app-pager"]'):
                    if page == 0:
                        with open('./vehicle.html', 'w', encoding='UTF-8') as file:
                            file.write(str(soup))
                    break
        except KeyboardInterrupt:
            break
        except:
            continue

    # = = = = = = = = = = = = = = =

    next_ = html.xpath('//a[@id="pagingBottom_nextButton"]')
    if page == 0 and next_ and next_[0].xpath('./@href')[0].strip() == 'Search/#':
        break

    # = = = = = = = = = = = = = = =

    list_text, list_tr = [text.strip() for text in html.xpath('//th[@scope="col"]/text()')], html.xpath('//tr[@class="detail-app-row"]')

    # = = = = = = = = = = = = = = =

    list_dict_correct_temp_temp = [{'Vehicle Url': crawler_series['Vehicle Url'],
                                    'Page': page + 1,
                                    'Row': i + 1} for i in range(len(list_tr))]

    for i, tr in enumerate(list_tr):
        for j, text in enumerate(list_text):
            list_dict_correct_temp_temp[i][text] = tr.xpath(f'./td[{j+1}]/text()')[0].strip()

    list_dict_correct_temp.extend(list_dict_correct_temp_temp)

    # = = = = = = = = = = = = = = =

    if not next_:
        break

    # = = = = = = = = = = = = = = =

    page += 1

list_dict_correct_temp

1 1
1 2


[{'Vehicle Url': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=45360&PartType=Brake%20Backing%20Plate',
  'Page': 1,
  'Row': 1,
  'Year': '2012',
  'Make': 'Ford',
  'Model': 'F-350 (Mexico)',
  'Configuration': 'Mexico Region',
  'Positions': 'Rear Left; Rear Right',
  'Application Notes': 'w/Dual Rear Wheels (DRW); No Hole for ABS Sensor; Left Caliper Mounted Rearward of Axle'},
 {'Vehicle Url': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=45360&PartType=Brake%20Backing%20Plate',
  'Page': 1,
  'Row': 2,
  'Year': '2012',
  'Make': 'Ford',
  'Model': 'F-350 Super Duty',
  'Configuration': '',
  'Positions': 'Rear Left; Rear Right',
  'Application Notes': 'w/Dual Rear Wheels (DRW); No Hole for ABS Sensor; Left Caliper Mounted Rearward of Axle'},
 {'Vehicle Url': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=45360&PartType=Brake%20Backing%20Plate',
  'Page': 1,
  'Row': 3,
  'Year': '2012',
  'Make': 'Ford',
  'Model': 'F-450 Super Duty',


In [6]:
list_dict_correct_temp

[{'Vehicle Url': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=45360&PartType=Brake%20Backing%20Plate',
  'Page': 1,
  'Row': 1,
  'Year': '2012',
  'Make': 'Ford',
  'Model': 'F-350 (Mexico)',
  'Configuration': 'Mexico Region',
  'Positions': 'Rear Left; Rear Right',
  'Application Notes': 'w/Dual Rear Wheels (DRW); No Hole for ABS Sensor; Left Caliper Mounted Rearward of Axle'},
 {'Vehicle Url': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=45360&PartType=Brake%20Backing%20Plate',
  'Page': 1,
  'Row': 2,
  'Year': '2012',
  'Make': 'Ford',
  'Model': 'F-350 Super Duty',
  'Configuration': '',
  'Positions': 'Rear Left; Rear Right',
  'Application Notes': 'w/Dual Rear Wheels (DRW); No Hole for ABS Sensor; Left Caliper Mounted Rearward of Axle'},
 {'Vehicle Url': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=45360&PartType=Brake%20Backing%20Plate',
  'Page': 1,
  'Row': 3,
  'Year': '2012',
  'Make': 'Ford',
  'Model': 'F-450 Super Duty',


In [7]:
list_dict_correct.extend(list_dict_correct_temp)

list_dict_correct

[{'Vehicle Url': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=45360&PartType=Brake%20Backing%20Plate',
  'Page': 1,
  'Row': 1,
  'Year': '2012',
  'Make': 'Ford',
  'Model': 'F-350 (Mexico)',
  'Configuration': 'Mexico Region',
  'Positions': 'Rear Left; Rear Right',
  'Application Notes': 'w/Dual Rear Wheels (DRW); No Hole for ABS Sensor; Left Caliper Mounted Rearward of Axle'},
 {'Vehicle Url': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=45360&PartType=Brake%20Backing%20Plate',
  'Page': 1,
  'Row': 2,
  'Year': '2012',
  'Make': 'Ford',
  'Model': 'F-350 Super Duty',
  'Configuration': '',
  'Positions': 'Rear Left; Rear Right',
  'Application Notes': 'w/Dual Rear Wheels (DRW); No Hole for ABS Sensor; Left Caliper Mounted Rearward of Axle'},
 {'Vehicle Url': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=45360&PartType=Brake%20Backing%20Plate',
  'Page': 1,
  'Row': 3,
  'Year': '2012',
  'Make': 'Ford',
  'Model': 'F-450 Super Duty',


In [8]:
crawler_series['Request_Url'] = url_request

list_series_error.append(crawler_series)

list_series_error

[No                                                                     1
 Type                                                 Brake Backing Plate
 Part Number                                                      924-212
 Brand                                              Dorman - OE Solutions
 Description                                 Brake Backing Plate - 1 Pair
 Application Summary                                         Ford 2012-05
 OEM                    5C3Z-2B636-AA;5C3Z-2B636-BA;8C3Z-2209-A;8C3Z-2...
 Picture                                                                 
 Url                    https://www.dormanproducts.com/p-45360-924-212...
 Json_Src               {"0": "https://static.dormanproducts.com/image...
 Vehicle Url            https://www.dormanproducts.com/itemdetailapp.a...
 Json_Specification     {"0": {"Brake Type": "Disc"}, "1": {"Color": "...
 Request_Url            https://www.dormanproducts.com/itemdetailapp.a...
 Name: 0, dtype: object]

In [9]:
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Vehicle Url', 'Page', 'Row'],
                                                             ascending=[True, True, True],
                                                             ignore_index=True)
    df_correct['Year'] = df_correct['Year'].astype(int)
    df_correct.to_excel('./test_vehicle.xlsx', index=False)

df_correct

,Vehicle Url,Page,Row,Year,Make,Model,Configuration,Positions,Application Notes
0,https://www.dormanproducts.com/itemdetailapp.a...,1,1,2012,Ford,F-350 (Mexico),Mexico Region,Rear Left; Rear Right,w/Dual Rear Wheels (DRW); No Hole for ABS Sens...
1,https://www.dormanproducts.com/itemdetailapp.a...,1,2,2012,Ford,F-350 Super Duty,,Rear Left; Rear Right,w/Dual Rear Wheels (DRW); No Hole for ABS Sens...
2,https://www.dormanproducts.com/itemdetailapp.a...,1,3,2012,Ford,F-450 Super Duty,Crew Cab Pickup,Rear Left; Rear Right,No Hole for ABS Sensor; Left Caliper Mounted R...
3,https://www.dormanproducts.com/itemdetailapp.a...,1,4,2011,Ford,F-350 (Mexico),Mexico Region,Rear Left; Rear Right,w/Dual Rear Wheels (DRW); No Hole for ABS Sens...
4,https://www.dormanproducts.com/itemdetailapp.a...,1,5,2011,Ford,F-350 Super Duty,,Rear Left; Rear Right,w/Dual Rear Wheels (DRW); No Hole for ABS Sens...
5,https://www.dormanproducts.com/itemdetailapp.a...,1,6,2011,Ford,F-450 Super Duty,Crew Cab Pickup,Rear Left; Rear Right,No Hole for ABS Sensor; Left Caliper Mounted R...
6,https://www.dormanproducts.com/itemdetailapp.a...,1,7,2010,Ford,F-250 Super Duty,,Rear Left; Rear Right,No Hole for ABS Sensor; Left Caliper Mounted R...
7,https://www.dormanproducts.com/itemdetailapp.a...,1,8,2010,Ford,F-350 Super Duty,,Rear Left; Rear Right,w/Dual Rear Wheels (DRW); No Hole for ABS Sens...
8,https://www.dormanproducts.com/itemdetailapp.a...,1,9,2009,Ford,F-250 Super Duty,,Rear Left; Rear Right,No Hole for ABS Sensor; Left Caliper Mounted R...
9,https://www.dormanproducts.com/itemdetailapp.a...,1,10,2009,Ford,F-350 (Mexico),Mexico Region,Rear Left; Rear Right,w/Dual Rear Wheels (DRW); No Hole for ABS Sens...


In [10]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_vehicle_error.xlsx', index=False)

df_error

,No,Type,Part Number,Brand,Description,Application Summary,OEM,Picture,Url,Json_Src,Vehicle Url,Json_Specification,Request_Url
0,1,Brake Backing Plate,924-212,Dorman - OE Solutions,Brake Backing Plate - 1 Pair,Ford 2012-05,5C3Z-2B636-AA;5C3Z-2B636-BA;8C3Z-2209-A;8C3Z-2...,,https://www.dormanproducts.com/p-45360-924-212...,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Brake Type"": ""Disc""}, ""1"": {""Color"": ""...",https://www.dormanproducts.com/itemdetailapp.a...
